# 뉴스 크롤링

In [ ]:
# 모듈 설치
!pip install selenium
!pip install webdriver-manager
!apt-get update
!pip install konlpy
# !apt install chromium-chromedriver

In [2]:
# import
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.cluster import DBSCAN

# DBSCAN : 밀도 기반 클러스터링
# 점 p에서 부터 거리 e (epsilon)내에 점이 m(minPts) 개 있으면 하나의 군집으로 인식

# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [3]:
# KST = timezone('Asia/Seoul')    # 서울 시간

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
# browser.implicitly_wait(1)

## 기사 링크 크롤링

In [14]:
class UrlCrawling:
    def __init__(self, six_url, entertainment_url, sports_url, category, category_names):
        self.six_url = six_url
        self.entertainment_url = entertainment_url
        self.sports_url = sports_url
        self.category = category
        self.category_names = category_names

    def getSixLink(self):    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학

        for category in range(6):     # 6
            a_list = []
            for page in range(1, 2):  # 1, 6
                url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
                browser.get(url)

                time.sleep(0.5)

                soup = BeautifulSoup(browser.page_source, "html.parser")
                a_list.extend(soup.select(".type06_headline dt+dt a"))
                a_list.extend(soup.select(".type06 dt+dt a"))

                print(f"{self.category_names[category]} {page} 페이지")

            for a in a_list:
                self.six_url.append(a["href"])
                self.category.append(self.category_names[category])


    def getEntertainmentLink(self):   # 연예
        # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
        a_list = []
        today = datetime.date.today()

        for page in range(1, 2):  # 1, 5
            url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            a_list.extend(soup.select(".news_lst li>a"))


            print(f"연예 {page} 페이지")

        for a in a_list:
            self.entertainment_url.append("https://entertain.naver.com" + a["href"])
            self.category.append(self.category_names[6])


    def getSportsLink(self):    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
        # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
        a_list = []
        today = str(datetime.date.today()).replace('-', '')

        for page in range(1, 2):  # 1, 7
            url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".news_list li>a"))

            print(f"스포츠 {page} 페이지")

        for i in range(len(a_list)):
            if i == 100:  # 100개 링크 추가했으면 멈추기
                break
            self.sports_url.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
            self.category.append(self.category_names[7])

url_crawler = UrlCrawling([], [], [], [], ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"])

url_crawler.getEntertainmentLink()

연예 1 페이지


In [15]:
print("6개 카테고리 수집 개수 :", len(url_crawler.six_url))
print("6개 카테고리 수집 개수 :", len(set(url_crawler.six_url)))
print()
print("연예 카테고리 수집 개수 :", len(url_crawler.entertainment_url))
print("연예 카테고리 수집 개수 :", len(set(url_crawler.entertainment_url)))
print()
print("스포츠 카테고리 수집 개수 :", len(url_crawler.sports_url))
print("스포츠 카테고리 수집 개수 :", len(set(url_crawler.sports_url)))
print()
print("전체 카테고리 수집 개수 : ", len(url_crawler.category))

6개 카테고리 수집 개수 : 0
6개 카테고리 수집 개수 : 0

연예 카테고리 수집 개수 : 25
연예 카테고리 수집 개수 : 25

스포츠 카테고리 수집 개수 : 0
스포츠 카테고리 수집 개수 : 0

전체 카테고리 수집 개수 :  25


## 제목, 본문, 날짜 크롤링

In [16]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
    article = re.sub('\[.{1,15}\]','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\w{2,4}기자','',article)
    article = re.sub('\w+ 기상캐스터','',article)
    article = re.sub('사진','',article)
    article = re.sub('포토','',article)

    article = re.sub('\S+@[a-z.]+','',article)          # 이메일 삭제

    article = re.sub('\n','',article)
    article = re.sub('\t','',article)
    article = re.sub('\u200b','',article)
    article = re.sub('\xa0','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    # article = re.sub('([a-zA-Z])','',article)   # 영어 삭제
    # article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘’“”|\(\)\[\]\<\>`\'…》]','',article)   # 특수문자 삭제

    return article


In [20]:
# 기사 본문 크롤링

class ContentCrawling:
    def __init__(self, title, content, date):
        self.title = title
        self.content = content
        self.date = date

    def getSixContent(self, url_list):   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
        title_list = []
        content_list = []
        date_list = []

        for url in url_list:
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            try:
                title_list.extend(soup.select("#title_area span"))              # 제목

                c = soup.find_all(attrs={"id" : "dic_area"})                    # 본문

                while c[0].find(attrs={"class" : "end_photo_org"}):             # 이미지 있는 만큼
                    c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

                while c[0].find(attrs={"class" : "vod_player_wrap"}):           # 영상 있는 만큼
                    c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

                if c[0].find(attrs={"class" : "artical-btm"}):                  # 하단에 제보하기 칸 있으면 삭제
                    c[0].find(attrs={"class" : "artical-btm"}).decompose()

                content_list.extend(c)

                date_list.extend(soup.select("._ARTICLE_DATE_TIME"))            # 날짜

            except IndexError:
                print("삭제된 기사")

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            self.date.append(d.text)


    def getEntertainmentContent(self, url_list):    # 연예
        title_list = []
        content_list = []
        date_list = []

        for url in url_list:
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            try:
                title_list.extend(soup.select(".end_tit"))                      # 제목

                c = soup.find_all(attrs={"class" : "article_body"})             # 본문

                while c[0].find(attrs={"class" : "end_photo_org"}):             # 이미지 있는 만큼
                    c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

                if c[0].find(attrs={"class" : "caption"}):                      # 이미지 설명 없애기
                    c[0].find(attrs={"class" : "caption"}).decompose()

                while c[0].find(attrs={"class" : "video_area"}):                # 영상 있는 만큼
                    c[0].find(attrs={"class" : "video_area"}).decompose()       # 본문 영상에 있는 글자 없애기

                while c[0].find(attrs={"name" : "iframe"}):
                    c[0].find(attrs={"name" : "iframe"}).decompose()

                content_list.extend(c)

                date_list.extend(soup.select_one(".author em"))                 # 날짜

            except IndexError:
                print("삭제된 기사")

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            self.date.append(d.text)


    def getSportsContent(self, url_list):   # 스포츠
        title_list = []
        content_list = []
        date_list = []

        for url in url_list:

            browser.get(url)                                                    # 스포츠 기사만 여기서 넘어가는데 굉장히 진짜 대박 오래 걸림 왜 그럴까? 스포츠
            print(url)
            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")

            title_list.extend(soup.select(".news_headline .title"))             # 제목

            c = soup.find_all(attrs={"class" : "news_end"})                     # 본문

            while c[0].find(attrs={"class" : "end_photo_org"}):                 # 이미지 있는 만큼
                c[0].find(attrs={"class" : "end_photo_org"}).decompose()        # 본문 이미지에 있는 글자 없애기

            while c[0].find(attrs={"class" : "image"}):
                c[0].find(attrs={"class" : "image"}).decompose()

            while c[0].find(attrs={"class" : "vod_area"}):                      # 영상 있는 만큼
                c[0].find(attrs={"class" : "vod_area"}).decompose()             # 본문 영상에 있는 글자 없애기

            c[0].find(attrs={"class" : "source"}).decompose()
            c[0].find(attrs={"class" : "byline"}).decompose()
            c[0].find(attrs={"class" : "reporter_area"}).decompose()
            c[0].find(attrs={"class" : "copyright"}).decompose()
            c[0].find(attrs={"class" : "categorize"}).decompose()
            c[0].find(attrs={"class" : "promotion"}).decompose()

            content_list.extend(c)

            date_list.extend(soup.select_one(".info span"))                     # 날짜

        for t in title_list:
            self.title.append(clean(t.text))

        for c in content_list:
            self.content.append(clean(c.text))

        for d in date_list:
            self.date.append(d.text)

    def makeDataFrame(self):    # 수집한 데이터를 데이터프레임으로 변환
        all_url = url_crawler.six_url + url_crawler.entertainment_url + url_crawler.sports_url

        article_df = pd.DataFrame({"category" : url_crawler.category,
                                   "date" : self.date,
                                   "title" : self.title,
                                   "content" : self.content,
                                   "url" : all_url})

        return article_df


content_crawler = ContentCrawling([], [], [])
content_crawler.getEntertainmentContent(url_crawler.entertainment_url)

In [ ]:
article_df = content_crawler.makeDataFrame()
article_df

In [22]:
print(len(content_crawler.title))
print(len(content_crawler.content))
print(len(content_crawler.date))

25
25
25


# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class


In [ ]:
def getNouns():
    okt = Okt()
    nouns_list = []   # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

    article_df["nouns"] = nouns_list   # 데이터 프레임에 추가


getNouns()

NameError: ignored

In [ ]:
# 영어 기사 삭제하기 위해 명사가 10개 미만이면 삭제

article_df.drop(article_df[article_df['nouns'].apply(lambda x : len(x) < 10)].index, inplace=True)
article_df.reset_index(drop=True, inplace=True)

In [ ]:
article_df

In [ ]:
# article_df.to_csv("C:\\Users\\82106\\OneDrive\\바탕 화면\\article.csv", encoding='utf-8-sig')

In [ ]:
# 명사까지 추출한 파일 불러오기, 1400개 기사

test_df = pd.read_csv("https://raw.githubusercontent.com/Yoon-juhan/naverNewsCrawling/main/article_2.csv")
test_df['nouns'] = test_df['nouns'].apply(lambda x: eval(x))        # 명사 열을 다시 리스트 형식으로 변환
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)  # 파일 불러왔을 때 필요없이 생기는 열 삭제

In [ ]:
test_df

In [ ]:
# category_names = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학", "연예", "스포츠"]

vector_list = []

def getVector():    # 카테고리 별로 벡터 생성
    for i in range(7):
        text = [" ".join(noun) for noun in test_df[test_df['category'] == category_names[i]]['nouns']]  # 명사 열을 하나의 리스트에 담는다.

        tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 5))
        tfidf_vectorizer.fit(text)
        vector = tfidf_vectorizer.transform(text).toarray()  # vector list 반환
        vector = np.array(vector)
        vector_list.append(vector)

getVector()

In [ ]:
len(vector_list)

7

# 군집화

**DBSCAN**
- https://bcho.tistory.com/1205


In [ ]:
# 카테고리 별로 군집화
# cluster_number 열에 군집 번호 생성

cluster_number_list = []

for vector in vector_list:
    model = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
    result = model.fit_predict(vector)
    cluster_number_list.extend(result)

test_df['cluster_number'] = cluster_number_list

In [ ]:
# 카테고리 별로 군집의 개수를 센다.
# 코드 수정하기

cluster_counts_df = pd.DataFrame({'category' : [""],
                                  'cluster_number' : [0],
                                  'cluster_count' : [0]})

for i in range(7):
    t = test_df[test_df['category'] == category_names[i]]['cluster_number'].value_counts().reset_index()
    t.columns = ['cluster_number', 'cluster_count']
    t['category'] = [category_names[i]] * len(t)

    cluster_counts_df = pd.concat([cluster_counts_df, t])

cluster_counts_df = cluster_counts_df[cluster_counts_df['cluster_count'] != 0]
cluster_counts_df

,category,cluster_number,cluster_count
0,정치,30,3
1,정치,69,2
2,정치,93,2
3,정치,0,1
4,정치,134,1
...,...,...,...
138,연예,44,1
139,연예,43,1
140,연예,42,1
141,연예,39,1


In [ ]:
# 상위 군집 10개씩만 추출

cluster_counts_df = cluster_counts_df[cluster_counts_df.index < 10]
cluster_counts_df

,category,cluster_number,cluster_count
0,정치,30,3
1,정치,69,2
2,정치,93,2
3,정치,0,1
4,정치,134,1
...,...,...,...
5,연예,28,4
6,연예,40,4
7,연예,62,4
8,연예,51,3


In [ ]:
# 군집화된 기사

clustered_article = pd.DataFrame()

for i in range(len(cluster_counts_df)):
    category_name, cluster_number = cluster_counts_df.iloc[i, 0:2]    # 카테고리 이름, 군집 번호

    clustered_article = pd.concat([clustered_article, test_df[(test_df['category'] == category_name) & (test_df['cluster_number'] == cluster_number)]])

clustered_article

# 요약

In [ ]:
# from gensim.summarization.summarizer import summarize

# txt = """."""

# result = summarize(txt, ratio=0.3)
# print(result)

# text (String, 필수) : 요약하고자하는 텍스트 원문
# ratio (Float, 선택적) : 텍스트 원문에 대한 요약 비율
# word_count (Int, 선택적) : 요약문에 포함될 단어의 수. ratio 인자와 같이 사용시 word_count 인자 우선함
# spilt (Bool, 선택적) : True로 전달하면 list 형태로 반환, False로 전달하면 String으로 반환